# SQL Asset - Partitioned

## Basic setup

In [4]:
# imports

import os
import great_expectations as gx
from great_expectations.core.partitioners import PartitionerYearAndMonth
import great_expectations.expectations as gxe
from great_expectations.checkpoint.v1_checkpoint import Checkpoint
from great_expectations.core.validation_definition import ValidationDefinition
from constants import DB_CONNECTION_STRING, TABLE_NAME

In [5]:
# constants

asset_name = "trip-data"
batch_definition_name = "partition-by-month"
checkpoint_name = "my_checkpoint"
datasource_name = "postgres"
suite_name = "passenger_count_checker"

In [6]:
# models

context = gx.get_context(mode="ephemeral")

datasource = context.sources.add_postgres(datasource_name, connection_string=DB_CONNECTION_STRING)
data_asset = datasource.add_table_asset(name=asset_name, table_name=TABLE_NAME)
batch_def = data_asset.add_batch_definition(batch_definition_name)
suite = context.add_expectation_suite(suite_name)
suite.add_expectation(
    gxe.ExpectColumnValuesToBeBetween(
        column="passenger_count", min_value=0, max_value=10
    )
)


ExpectColumnValuesToBeBetween(id='27dd79dc-3a8c-4218-9f70-17f7c8b88851', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, catch_exceptions=True, rendered_content=None, batch_id=None, row_condition=None, condition_parser=None, mostly=1.0, column='passenger_count', min_value=0.0, max_value=10.0, strict_min=False, strict_max=False)

## Check that we just get one batch (that contains everything)

In [7]:
batch_request = batch_def.build_batch_request()
batches = data_asset.get_batch_list_from_batch_request(batch_request)

# verify that we have exactly one batch
print([b.id for b in batches])

['postgres-trip-data']


## Run ValidationDefinition

In [8]:
validation_definition = ValidationDefinition(name="epheemeral validation definition", data=batch_def, suite=suite)

results = validation_definition.run()
print(results)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "postgres-trip-data",
          "column": "passenger_count",
          "min_value": 0.0,
          "max_value": 10.0
        },
        "meta": {},
        "id": "27dd79dc-3a8c-4218-9f70-17f7c8b88851"
      },
      "result": {
        "element_count": 3800,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 150,
        "missing_percent": 3.9473684210526314,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_counts": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    }
  ],
  "suite_name": "passenger_count_checker",
  "evalu

## Create and run a Checkpoint

In [9]:
# We can also run this in a checkpoint
checkpoint = context.checkpoints.add(Checkpoint(
    name=checkpoint_name,
    validation_definitions=[
        ValidationDefinition(name="validation_def", data=batch_def, suite=suite)
    ],
    actions=[],
))

results = checkpoint.run()
print(results)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

run_id={"run_name": null, "run_time": "2024-04-12T16:32:19.941888-04:00"} run_results={ValidationResultIdentifier::passenger_count_checker/__none__/20240412T203219.941888Z/postgres-trip-data: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "postgres-trip-data",
          "column": "passenger_count",
          "min_value": 0.0,
          "max_value": 10.0
        },
        "meta": {},
        "id": "27dd79dc-3a8c-4218-9f70-17f7c8b88851"
      },
      "result": {
        "element_count": 3800,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 150,
        "missing_percent": 3.9473684210526314,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_counts": []
      },
      "meta": {},
      "exc